# Fine-tuning Cross-Encoders

In [2]:
from typing import List
from llama_index.finetuning.cross_encoders.cross_encoder import (
    CrossEncoderFinetuneEngine,
    CrossEncoderFinetuningDatasetSample
)
import os
import pandas as pd

data_folder = "data"

version = 1

df_train = pd.read_csv(os.path.join(data_folder, f"train_{version}.csv"), index_col=0)
df_test = pd.read_csv(os.path.join(data_folder, f"test_{version}.csv"), index_col=0)
df_finetuning = pd.read_csv(os.path.join(data_folder, f"fine_tuning_{version}.csv"), index_col=0)

finetuning_dataset: List[CrossEncoderFinetuningDatasetSample] = []

for _, row in df_finetuning.iterrows():
    finetuning_dataset.append(
        CrossEncoderFinetuningDatasetSample(
            query=row["question"],
            context=row["context"],
            score=row["score"]
        )
    )

finetuning_engine = CrossEncoderFinetuneEngine(
    dataset=finetuning_dataset, epochs=2, batch_size=8, model_output_path="../models/fine_tuned_cross_encoder"
)

# Finetune the cross-encoder model
finetuning_engine.finetune()

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/300 [00:00<?, ?it/s]

Iteration:   0%|          | 0/300 [00:00<?, ?it/s]

# Evaluate Reranking

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
# Load Reranking Dataset
import pandas as pd
import os
import ast

data_folder = "data"
version = 1

df_reranking = pd.read_csv(os.path.join(data_folder, f"reranking_test_{version}.csv"), index_col=0)
df_reranking["questions"] = df_reranking["questions"].apply(ast.literal_eval)
df_reranking["context"] = df_reranking["context"].apply(ast.literal_eval)

In [ ]:
df_reranking.head()

In [ ]:
import openai

os.environ["OPENAI_API_KEY"] = "sk-Ei89D8xbv2ve98FT6e629a55Ca1341518f1238BaB53dAf20"
os.environ["OPENAI_API_BASE"] = "https://ai-yyds.com/v1"
openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_base = os.environ["OPENAI_API_BASE"]

In [ ]:
# We evaluate by calculating hits for each (question, context) pair,
# we retrieve top-k documents with the question, and
# it’s a hit if the results contain the context
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
)
from llama_index import Document

service_context_reranker_eval = ServiceContext.from_defaults(chunk_size=256, llm="local", embed_model="local")
rerank_base = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-12-v2", top_n=3
)

rerank_finetuned = SentenceTransformerRerank(
    model="../models/cross_encoder", top_n=3
)

In [ ]:
without_reranker_hits = 0
base_reranker_hits = 0
finetuned_reranker_hits = 0
total_number_of_context = 0
for index, row in df_reranking.iterrows():
    documents = [Document(text=row["paper"])]
    query_list = row["questions"]
    context_list = row["context"]

    assert len(query_list) == len(context_list)
    vector_index = VectorStoreIndex.from_documents(
        documents, service_context=service_context_reranker_eval
    )

    retriever_without_reranker = vector_index.as_query_engine(
        similarity_top_k=3, response_mode="no_text"
    )
    retriever_with_base_reranker = vector_index.as_query_engine(
        similarity_top_k=8,
        response_mode="no_text",
        node_postprocessors=[rerank_base],
    )
    retriever_with_finetuned_reranker = vector_index.as_query_engine(
        similarity_top_k=8,
        response_mode="no_text",
        node_postprocessors=[rerank_finetuned],
    )

    for index in range(0, len(query_list)):
        query = query_list[index]
        context = context_list[index]
        total_number_of_context += 1

        response_without_reranker = retriever_without_reranker.query(query)
        without_reranker_nodes = response_without_reranker.source_nodes

        for node in without_reranker_nodes:
            if context in node.node.text or node.node.text in context:
                without_reranker_hits += 1

        response_with_base_reranker = retriever_with_base_reranker.query(query)
        with_base_reranker_nodes = response_with_base_reranker.source_nodes

        for node in with_base_reranker_nodes:
            if context in node.node.text or node.node.text in context:
                base_reranker_hits += 1

        response_with_finetuned_reranker = (
            retriever_with_finetuned_reranker.query(query)
        )
        with_finetuned_reranker_nodes = (
            response_with_finetuned_reranker.source_nodes
        )

        for node in with_finetuned_reranker_nodes:
            if context in node.node.text or node.node.text in context:
                finetuned_reranker_hits += 1
        
        assert (
            len(with_finetuned_reranker_nodes)
            == len(with_base_reranker_nodes)
            == len(without_reranker_nodes)
        )

### Result
As we can see below we get more hits with finetuned_cross_encoder compared to other options.

In [ ]:
without_reranker_scores = [without_reranker_hits]
base_reranker_scores = [base_reranker_hits]
finetuned_reranker_scores = [finetuned_reranker_hits]
reranker_eval_dict = {
    "Metric": "Hits",
    "BGE_Embeddings": without_reranker_scores,
    "Base_cross_encoder": base_reranker_scores,
    "Finetuned_cross_encoder": finetuned_reranker_hits,
    "Total Relevant Context": total_number_of_context,
}
df_reranker_eval_results = pd.DataFrame(reranker_eval_dict)
display(df_reranker_eval_results)

## RAG Evaluation

In [1]:
import pandas as pd
import os
import ast

data_folder = "data"

version = 1

df_test = pd.read_csv(os.path.join(data_folder, f"test_{version}.csv"), index_col=0)
df_test["questions"] = df_test["questions"].apply(ast.literal_eval)
df_test["answers"] = df_test["answers"].apply(ast.literal_eval)
print(f"Number of papers in the test sample:- {len(df_test)}")

Number of papers in the test sample:- 80


In [2]:
df_test.head()

,paper,questions,answers
0,Identifying Condition-Action Statements in Med...,[What supervised machine learning models do th...,"[Unacceptable, Unacceptable, 1470 sentences, U..."
1,Multilingual is not enough: BERT for Finnish\t...,[By how much did the new model outperform mult...,"[For POS, improvements for cased BERT are 1.2..."
2,Efficient Dynamic WFST Decoding for Personaliz...,"[What does the cache consist of?, What languag...","[Unacceptable, Unacceptable, A model that cont..."
3,"A system for the 2019 Sentiment, Emotion and C...",[Did they pre-train on existing sentiment corp...,"[Unacceptable, Unacceptable, 2, Unacceptable]"
4,Towards Automatic Bot Detection in Twitter for...,[Do the authors report results on only English...,"[Unacceptable, Unacceptable, An existing bot d..."


### Baseline Evaluation
Use BAAI/bge-small-en as the base model for RAG without reranker

#### Eval Method:-
1. Iterate over each row of the test dataset:-
    1. For the current row being iterated, create a vector index using the paper document provided in the paper column of the dataset
    2. Query the vector index with a top_k value of top 3 nodes without any reranker
    3. Compare the generated answers with the reference answers of the respective sample using Pairwise Comparison Evaluator and add the scores to a list
5. Repeat 1 untill all the rows have been iterated
6. Calculate avg scores over all samples/ rows


In [3]:
from qasper_data.data import AnswerType
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    Document,
)


import openai



os.environ["OPENAI_API_KEY"] = "sk-Ei89D8xbv2ve98FT6e629a55Ca1341518f1238BaB53dAf20"
os.environ["OPENAI_API_BASE"] = "https://ai-yyds.com/v1"
openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_base = os.environ["OPENAI_API_BASE"]

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
from ragas import evaluate
from datasets import Dataset

from qasper_data.qasper_dataset import QasperDataset
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(chunk_size=256, llm="local", embed_model="local")

test_dataset = QasperDataset("test")

df_all_papers = pd.DataFrame()

df_test = df_test.head(10)

for index, row in df_test.iterrows():
    documents = [Document(text=row["paper"])]
    query_list = row["questions"]
    reference_answers_list = list(map(lambda x: x.split(","), row["answers"]))
    vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)
    contexts = []
    title = row["paper"].split("\t")[0]
    paper = test_dataset.match_paper_by_title(title) 
    for qa in paper.qas:
        context = []
        for answer in qa.answers:
            if answer.answer_type == AnswerType.EXTRACTIVE:
                spans = ""
                for span in answer.extractive_spans:
                    spans += span + "\t"
                context.append(spans)
        contexts.append(context)        
    print(contexts)
    # Query the vector index with a top_k value of top 3 documents without any reranker
    query_engine = vector_index.as_query_engine(similarity_top_k=3)
    response_list = []
    for query in query_list:
        response = query_engine.query(query)
        response_list.append(response.response)
    # Compare the generated answers with the reference answers of the respective sample using Ragas Metrics
    # and add the scores to a list
    dataset = Dataset.from_dict(
        {
            "question": query_list,
            "answer": response_list,
            "contexts": contexts,
            "ground_truths": reference_answers_list
        }
    )
    ragas_result = evaluate(
        dataset,
        metrics=[
            answer_relevancy,
            faithfulness,
            context_recall,
            context_precision,
        ],
    )
    df_ragas = ragas_result.to_pandas()
    df_all_papers = pd.concat([df_all_papers, df_ragas], ignore_index=True)

df_all_papers.to_csv(os.path.join(data_folder, f"ragas_baseline_{version}.csv"))

from IPython.display import clear_output

clear_output()

    

Found cached dataset qasper (/Users/zeyuli/.cache/huggingface/datasets/allenai___qasper/qasper/0.3.0/2bfcd239e581ab83f9ab7b76a82e42c6bcf574a13246ae6cc5a6c357c35f96f9)


  0%|          | 0/3 [00:00<?, ?it/s]

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/zeyuli/Library/Caches/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q4_0     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q

[['ZeroR, Naïve Bayes, J48, and random forest classifiers\t', 'ZeroR, Naïve Bayes, J48, and random forest \t'], [], [], ["Conditions that affect erythrocyte turnover and hemoglobin variants must be considered, particularly when the A1C result does not correlate with the patient's clinical situation\t", 'If patients have asthma, then beta-blockers, including eye drops, are contraindicated\t']]



llama_print_timings:        load time =   17686.12 ms
llama_print_timings:      sample time =      11.37 ms /    74 runs   (    0.15 ms per token,  6508.36 tokens per second)
llama_print_timings: prompt eval time =   23019.76 ms /   916 tokens (   25.13 ms per token,    39.79 tokens per second)
llama_print_timings:        eval time =    5242.38 ms /    73 runs   (   71.81 ms per token,    13.92 tokens per second)
llama_print_timings:       total time =   28463.00 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =   17686.12 ms
llama_print_timings:      sample time =      30.24 ms /   129 runs   (    0.23 ms per token,  4265.87 tokens per second)
llama_print_timings: prompt eval time =    5058.27 ms /    19 tokens (  266.22 ms per token,     3.76 tokens per second)
llama_print_timings:        eval time =    8790.31 ms /   128 runs   (   68.67 ms per token,    14.56 tokens per second)
llama_print_timings:       total time =   14383.03 ms
Llama.generate: prefix-

...

evaluating with [context_precision]


100%|██████████| 1/1 [00:01<00:00,  1.80s/it]


[[], ['Seq2seq\tGPT2-FT\tSpeaker\tECM\tSkeleton-to-Response (SR)\tRetrieval + Style Transfer (RST)\tRetrieval + Reranking (RRe)\t', 'Generative Approaches ::: Seq2seq\tGenerative Approaches ::: GPT2-FT:\tGenerative Approaches ::: Speaker:\tGenerative Approaches ::: ECM:\tRetrieval-Based Approaches ::: Skeleton-to-Response (SR)\tRetrieval-Based Approaches ::: Retrieval + Style Transfer (RST)\tRetrieval-Based Approaches ::: Retrieval + Style Transfer (RST)\tRetrieval-Based Approaches ::: Retrieval + Reranking (RRe)\t'], ['Chinese\tEnglish\t'], ['gender-specific (Chinese) dataset\temotion-specific (Chinese) dataset\tsentiment-specific (English) dataset\t', 'Gender-Specific Dialogue Dataset\tEmotion-Specific Dialogue Dataset\tSentiment-Specific Dialogue Dataset\t']]


Llama.generate: prefix-match hit

llama_print_timings:        load time =   17686.12 ms
llama_print_timings:      sample time =      30.22 ms /   176 runs   (    0.17 ms per token,  5823.19 tokens per second)
llama_print_timings: prompt eval time =   14960.75 ms /   831 tokens (   18.00 ms per token,    55.55 tokens per second)
llama_print_timings:        eval time =   12476.74 ms /   175 runs   (   71.30 ms per token,    14.03 tokens per second)
llama_print_timings:       total time =   28126.10 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =   17686.12 ms
llama_print_timings:      sample time =      42.71 ms /   256 runs   (    0.17 ms per token,  5994.19 tokens per second)
llama_print_timings: prompt eval time =   11423.41 ms /   793 tokens (   14.41 ms per token,    69.42 tokens per second)
llama_print_timings:        eval time =   18488.57 ms /   255 runs   (   72.50 ms per token,    13.79 tokens per second)
llama_print_timings:       total time =   30

KeyboardInterrupt: 

In [ ]:
df_all_papers.head()

In [ ]:
def mean_score(df):
    return df.mean(axis=0)

answer_relevancy_score = mean_score(df_all_papers["answer_relevancy"])
faithfulness_score = mean_score(df_all_papers["faithfulness"])
context_recall_score = mean_score(df_all_papers["context_recall"])
context_precision_score = mean_score(df_all_papers["context_precision"])

ragas_baseline_dict = {
    "Metric": ["answer_relevancy", "faithfulness", "context_recall", "context_precision"],
    "BGE_Embeddings": [answer_relevancy_score, faithfulness_score, context_recall_score, context_precision_score],
}
df_ragas_baseline_results = pd.DataFrame(ragas_baseline_dict)

In [ ]:
df_ragas_baseline_results

### RAG Base Reranker Evaluation
`BAAI/bge-small-en` Embedding + `cross-encoder/ms-marco-MiniLM-L-12-v2` as reranker

#### Eval Method:-
1. Iterate over each row of the test dataset:-
    1. For the current row being iterated, create a vector index using the paper document provided in the paper column of the dataset
    2. Query the vector index with a top_k value of top 5 nodes.
    3. Use cross-encoder/ms-marco-MiniLM-L-12-v2 as a reranker as a NodePostprocessor to get top_k value of top 3 nodes out of the 8 nodes
    3. Compare the generated answers with the reference answers of the respective sample using Ragas Evaluator and add the scores to a list
5. Repeat 1 untill all the rows have been iterated
6. Calculate avg scores over all samples/ rows

In [ ]:
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import (VectorStoreIndex, ServiceContext, Document)

reranker = SentenceTransformerRerank(model="cross-encoder/ms-marco-MiniLM-L-12-v2", top_n=3)

import pandas as pd
import os
import openai
import nest_asyncio

nest_asyncio.apply()

os.environ["OPENAI_API_KEY"] = "sk-Ei89D8xbv2ve98FT6e629a55Ca1341518f1238BaB53dAf20"
os.environ["OPENAI_API_BASE"] = "https://ai-yyds.com/v1"

openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_base = os.environ["OPENAI_API_BASE"]

data_folder = "data"
version = 1

service_context = ServiceContext.from_defaults(chunk_size=256, llm="local", embed_model="local")

In [ ]:
from tqdm.notebook import tqdm

df_test = pd.read_csv(os.path.join(data_folder, f"test_{version}.csv"), index_col=0)

import ast

df_test["questions"] = df_test["questions"].apply(ast.literal_eval)
df_test["answers"] = df_test["answers"].apply(ast.literal_eval)

from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)

from ragas import evaluate

from datasets import Dataset


metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
]

df_ragas_base_reranker_result = pd.DataFrame()



for index, row in tqdm(df_test.iterrows()):
    documents = [Document(text=row["paper"])]
    query_list = row["questions"]
    reference_answers_list = row["answers"]
    response_list = []

    number_of_accepted_queries = 0
    
    # Create a vector index from the documents
    vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)
    
    # Query the vector index with a top_k value of top 8 nodes with reranker
    # as cross-encoder/ms-marco-MiniLM-L-12-v2
    query_engine = vector_index.as_query_engine(
        similarity_top_k=3, node_postprocessors=[reranker]
    )
    assert len(query_list) == len(reference_answers_list)
    
    reference_answers_list = list(map(lambda x: x.split(","), row["answers"]))
    for query in query_list:
        response = query_engine.query(query)
        response_list.append(response)
    
    answers = []
    contexts = []
    for r in response_list:
        answers.append(r.response)
        contexts.append([c.node.get_content() for c in r.source_nodes])
        
    dataset = Dataset.from_dict(
        {
            "question": query_list,
            "answer": answers,
            "contexts": contexts,
            "ground_truths": reference_answers_list
        }
    )
    
    ragas_result = evaluate(
        dataset,
        metrics=metrics,
    )
    
    df_ragas = ragas_result.to_pandas()
    df_ragas_base_reranker_result = pd.concat([df_ragas_base_reranker_result, df_ragas], ignore_index=True)

df_ragas_base_reranker_result.to_csv(os.path.join(data_folder, f"ragas_base_reranker_{version}.csv"))


from IPython.display import clear_output

clear_output()



In [ ]:
df_ragas_base_reranker_result.head()

In [ ]:
def mean_score(df):
    return df.mean(axis=0)

answer_relevancy_score = mean_score(df_ragas_base_reranker_result["answer_relevancy"])
faithfulness_score = mean_score(df_ragas_base_reranker_result["faithfulness"])
context_recall_score = mean_score(df_ragas_base_reranker_result["context_recall"])
context_precision_score = mean_score(df_ragas_base_reranker_result["context_precision"])

ragas_base_reranker_dict = {
    "Metric": ["answer_relevancy", "faithfulness", "context_recall", "context_precision"],
    "Base_Cross_Encoder": [answer_relevancy_score, faithfulness_score, context_recall_score, context_precision_score],
}
df_ragas_base_reranker_results = pd.DataFrame(ragas_base_reranker_dict)

In [ ]:
df_ragas_base_reranker_results

### Evaluate with Fine-Tuned re-ranker
`BAAI/bge-small-en` Embedding + `cross_encoder` fine-tuned on the fine-tuning dataset as reranker

#### Eval Method:-
1. Iterate over each row of the test dataset:-
    1. For the current row being iterated, create a vector index using the paper document provided in the paper column of the dataset
    2. Query the vector index with a top_k value of top 5 nodes.
    3. Use cross_encoder fine-tuned on the fine-tuning dataset as a reranker as a NodePostprocessor to get top_k value of top 3 nodes out of the 8 nodes
    3. Compare the generated answers with the reference answers of the respective sample using Ragas Evaluator and add the scores to a list

In [ ]:
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import (VectorStoreIndex, ServiceContext, Document)

reranker = SentenceTransformerRerank(model="../models/cross_encoder", top_n=3)

import pandas as pd
import os
import openai
import nest_asyncio

nest_asyncio.apply()

os.environ["OPENAI_API_KEY"] = "sk-Ei89D8xbv2ve98FT6e629a55Ca1341518f1238BaB53dAf20"
os.environ["OPENAI_API_BASE"] = "https://ai-yyds.com/v1"

openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_base = os.environ["OPENAI_API_BASE"]

data_folder = "data"
version = 1

service_context = ServiceContext.from_defaults(chunk_size=256, llm="local", embed_model="local")

In [ ]:
from tqdm.notebook import tqdm
from ragas.llama_index import evaluate

df_test = pd.read_csv(os.path.join(data_folder, f"test_{version}.csv"), index_col=0)

import ast

df_test["questions"] = df_test["questions"].apply(ast.literal_eval)
df_test["answers"] = df_test["answers"].apply(ast.literal_eval)

from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
]


from datasets import Dataset

df_ragas_finetuned_reranker_result = pd.DataFrame()

for index, row in tqdm(df_test.iterrows()):
    documents = [Document(text=row["paper"])]
    query_list = row["questions"]
    reference_answers_list = row["answers"]
    response_list = []

    number_of_accepted_queries = 0
    
    # Create a vector index from the documents
    vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)
    
    # Query the vector index with a top_k value of top 8 nodes with reranker
    # as cross-encoder/ms-marco-MiniLM-L-12-v2
    query_engine = vector_index.as_query_engine(
        similarity_top_k=3, node_postprocessors=[reranker]
    )
    assert len(query_list) == len(reference_answers_list)
    
    reference_answers_list = list(map(lambda x: x.split(","), row["answers"]))
    
    result = evaluate(query_engine, metrics, query_list, reference_answers_list)
    
    df_ragas = result.to_pandas()
    df_ragas_finetuned_reranker_result = pd.concat([df_ragas_finetuned_reranker_result, df_ragas], ignore_index=True)

df_ragas_finetuned_reranker_result.to_csv(os.path.join(data_folder, f"ragas_finetuned_reranker_{version}.csv"))

from IPython.display import clear_output
clear_output()

In [ ]:
df_ragas_finetuned_reranker_result.head()

In [ ]:
def mean_score(df):
    return df.mean(axis=0)

answer_relevancy_score = mean_score(df_ragas_finetuned_reranker_result["answer_relevancy"])
faithfulness_score = mean_score(df_ragas_finetuned_reranker_result["faithfulness"])
context_recall_score = mean_score(df_ragas_finetuned_reranker_result["context_recall"])
context_precision_score = mean_score(df_ragas_finetuned_reranker_result["context_precision"])

ragas_finetuned_reranker_dict = {
    "Metric": ["answer_relevancy", "faithfulness", "context_recall", "context_precision"],
    "Finetuned_Cross_Encoder": [answer_relevancy_score, faithfulness_score, context_recall_score, context_precision_score],
}

df_ragas_finetuned_reranker_results = pd.DataFrame(ragas_finetuned_reranker_dict)

In [ ]:
df_ragas_finetuned_reranker_results